In [ ]:
from __future__ import unicode_literals
import youtube_dl
import os
import pandas as pd

## Script for Youtube Download

In [ ]:
class MyLogger(object):
    def debug(self, msg):
        pass

    def warning(self, msg):
        pass

    def error(self, msg):
        print(msg)


def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

def download_clip(url, name, outdir):

    filename = os.path.join(outdir,name)

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': filename,
        'noplaylist': True,
        'continue_dl': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192', }],
        'logger': MyLogger(),
        'progress_hooks': [my_hook],
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.cache.remove()
            info_dict = ydl.extract_info(url, download=False)
            ydl.prepare_filename(info_dict)
            ydl.download([url])
            return True
    except Exception:
        return False

### Donwload youtube songs

In [ ]:
filelist = "list.csv"
outdir = "/mnt/dev/dirceusilva/dados/Cover/SHS100k"


In [ ]:
list_songs_df = pd.read_csv(filelist, sep='\t', names=["set_id", "ver_id", "title", "performer", "url", "status"], header=None)


In [ ]:
index_failed=[]
for k in range(107205,len(list_songs_df)): #
    row = list_songs_df.iloc[k]
    print("{:d} de {:d} - {} - {}".format(k, len(list_songs_df), row["set_id"], row["ver_id"]))
    dirout = os.path.join(outdir, str(row["set_id"]))
    
    try:
        os.mkdir(dirout)
    except:
        print("Creation of the directory {} failed".format(row["set_id"]))
        
    name = str(row["ver_id"])+ '.%(ext)s'
    if not download_clip(row["url"], name, dirout):
        index_failed.append(k)

In [ ]:
print(len(index_failed))


In [4]:
import pandas as pd

shs100k = pd.read_csv("list.csv", sep='\t', header=None)

In [6]:
shs100k.columns=["set_id", "ver_id", "title", "performer", "url", "status"]

In [9]:
test = pd.read_csv("SHS100K-TEST", sep='\t', header=None)
test.columns = ["set_id", "ver_id"]
train = pd.read_csv("SHS100K-TRAIN", sep='\t', header=None)
train.columns = ["set_id", "ver_id"]
val = pd.read_csv("SHS100K-VAL", sep='\t', header=None)
val.columns = ["set_id", "ver_id"]


In [11]:
train["subset"] = "TRAIN"
test["subset"] = "TEST"
val["subset"] = "VAL"



In [13]:
shs100k = pd.merge(
    pd.merge(
        pd.merge(shs100k, train, on=["set_id", "ver_id"], how='left'), val, on=["set_id", "ver_id"], how='left'), 
         test, on=["set_id", "ver_id"], how='left')

In [21]:
import numpy as np

def get_subset(x):
    if x.subset_x not in [None, np.nan, "NaN"]:
        return x.subset_x
    elif x.subset_y not in [None, np.nan, "NaN"]:
        return x.subset_y
    elif x.subset not in [None, np.nan, "NaN"]:
        return x.subset
    

shs100k["subset"] = shs100k.apply(lambda x: get_subset(x), axis=1)

In [24]:
shs100k = shs100k.drop(["subset_x", "subset_y"], axis=1)

In [51]:
train["joined_id"] = train["set_id"].astype(str) + '_' + train["ver_id"].astype(str)
test["joined_id"] = test["set_id"].astype(str) + '_' + test["ver_id"].astype(str)
val["joined_id"] = val["set_id"].astype(str) + '_' + val["ver_id"].astype(str)


def get_subset(x):
    
    joined_id = str(x.set_id) + '_' + str(x.ver_id)
    if joined_id in train["joined_id"].values:
        return "TRAIN"
    elif joined_id in test["joined_id"].values:
        return "TEST"
    elif joined_id in val["joined_id"].values:
        return "VAL"
    

In [52]:
shs100k["subset"] = shs100k.apply(lambda x: get_subset(x), axis=1)

In [58]:
shs100k.to_csv("shs100k.csv", sep=';')

In [ ]:
import pandas as pd

pd.read_csv("../da-ta")

In [ ]:
!ls ../da-tacos

In [66]:
shs100k["yt_id"] = shs100k["url"].apply(lambda x: x.split("?v=")[-1])

In [68]:
shs100k.drop("url", axis=1).to_csv("shs100k.csv", sep=';', index=None)

In [69]:
shs100k[shs100k.subset == "TEST"]

,set_id,ver_id,title,performer,url,status,subset,yt_id
2779,21,1,Yesterday,Marianne Faithfull,https://www.youtube.com/watch?v=gORyrU1xQpg,True,TEST,gORyrU1xQpg
2780,21,2,Yesterday,Matt Monro,https://www.youtube.com/watch?v=jQhC3bMMLmw,True,TEST,jQhC3bMMLmw
2781,21,3,Yesterday,Jan & Dean,https://www.youtube.com/watch?v=D8RS6zzghCQ,True,TEST,D8RS6zzghCQ
2782,21,4,Yesterday,Mary Wells,https://www.youtube.com/watch?v=Y1qYRFZfPm8,True,TEST,Y1qYRFZfPm8
2783,21,5,Yesterday,The Hollyridge Strings,https://www.youtube.com/watch?v=eI3-EVEU51s,True,TEST,eI3-EVEU51s
...,...,...,...,...,...,...,...,...
114143,9463,5,Ooby Dooby,Jerry Lee Lewis,https://www.youtube.com/watch?v=I6wJBwoxark,True,TEST,I6wJBwoxark
114144,9463,6,Ooby-Dooby,Charlie Feathers,https://www.youtube.com/watch?v=15zXVxJfuaQ,True,TEST,15zXVxJfuaQ
115330,9759,0,I'm Losing You,"John Lennon, Yoko Ono",https://www.youtube.com/watch?v=Bbb3ekGecuE,True,TEST,Bbb3ekGecuE
115331,9759,1,I'm Losing You,Triage,https://www.youtube.com/watch?v=5bHWya-f7D4,True,TEST,5bHWya-f7D4


In [75]:
df = shs100k.rename({"set_id": "W_ID", "ver_id": "P_ID"}, axis=1)

grouped = df.groupby('W_ID')

result = {}
for w_id, group in grouped:
    result[w_id] = {}
    for _, row in group.iterrows():
        p_id = row['P_ID']
        yt_id = row['yt_id']
        result[w_id][p_id] = yt_id

In [78]:
shs100k[shs100k.subset == "TEST"].to_csv("../data/SHS100K/shs100k_test.csv", index=None, sep=';')

In [81]:
#!ls ../data/SHS100K/features | wc -l

!ls ../data/SHS100K/features/NhdF6ni58fI.h5

ls: cannot access '../data/SHS100K/features/NhdF6ni58fI.h5': No such file or directory


In [83]:
shs100k

,set_id,ver_id,title,performer,url,status,subset,yt_id
0,0,0,Summertime,Abbie Mitchell,https://www.youtube.com/watch?v=TMHWEcSKEPI,True,TRAIN,TMHWEcSKEPI
1,0,1,Summertime,Helen Jepson,https://www.youtube.com/watch?v=E3lw8gaHs04,True,TRAIN,E3lw8gaHs04
2,0,2,Summertime,Billie Holiday and Her Orchestra,https://www.youtube.com/watch?v=aPpJPTJc1lU,True,TRAIN,aPpJPTJc1lU
3,0,3,Summertime,Jerry Kruger and Her Orchestra,https://www.youtube.com/watch?v=J9JctRqcDso,True,TRAIN,J9JctRqcDso
4,0,4,Overture and Summertime,Anne Brown,https://www.youtube.com/watch?v=R44waInkjgI,True,TRAIN,R44waInkjgI
...,...,...,...,...,...,...,...,...
116348,9994,1,Sytten ti,Otto Brandenburg,https://www.youtube.com/watch?v=EVsRN0eDGNc,True,VAL,EVsRN0eDGNc
116349,9996,0,We Want the Airwaves,Ramones,https://www.youtube.com/watch?v=VJ5X721ABs4,True,VAL,VJ5X721ABs4
116350,9998,0,I Don't Care,Ramones,https://www.youtube.com/watch?v=QveJv7V4OZI,True,VAL,QveJv7V4OZI
116351,9998,1,I Don't Care,The Flatmates,https://www.youtube.com/watch?v=6L8jlQfp3wA,True,VAL,6L8jlQfp3wA


In [85]:
import os

yt_ids = os.listdir("../data/SHS100K/features")

In [94]:
shs100k["downloaded"] = shs100k["yt_id"].apply(lambda x: x + '.h5' in yt_ids)

In [98]:
shs100k[(~shs100k.downloaded) & (shs100k.subset == "TEST")]

,set_id,ver_id,title,performer,url,status,subset,yt_id,downloaded
2781,21,3,Yesterday,Jan & Dean,https://www.youtube.com/watch?v=D8RS6zzghCQ,True,TEST,D8RS6zzghCQ,False
2782,21,4,Yesterday,Mary Wells,https://www.youtube.com/watch?v=Y1qYRFZfPm8,True,TEST,Y1qYRFZfPm8,False
2791,21,13,Yesterday,Bobby Goldsboro,https://www.youtube.com/watch?v=U2Ez8ZNPUoE,True,TEST,U2Ez8ZNPUoE,False
2801,21,23,Yesterday,Jack Jones,https://www.youtube.com/watch?v=UsdrK6k96rA,True,TEST,UsdrK6k96rA,False
2802,21,24,Yesterday,Perry Como,https://www.youtube.com/watch?v=Q9nKY0VzWOw,True,TEST,Q9nKY0VzWOw,False
...,...,...,...,...,...,...,...,...,...
104843,7460,0,Gold Dust Woman,Fleetwood Mac,https://www.youtube.com/watch?v=cMp57bUzOB8,True,TEST,cMp57bUzOB8,False
104850,7460,7,Gold Dust Woman,Karen Elson,https://www.youtube.com/watch?v=2hC_6niLNdY,True,TEST,2hC_6niLNdY,False
104852,7460,9,Gold Dust Woman,Warren Haynes featuring Railroad Earth and Gra...,https://www.youtube.com/watch?v=DbeAjB-RCQU,True,TEST,DbeAjB-RCQU,False
114144,9463,6,Ooby-Dooby,Charlie Feathers,https://www.youtube.com/watch?v=15zXVxJfuaQ,True,TEST,15zXVxJfuaQ,False


## Convert  Wav to MP3

In [ ]:
import glob

direct = '/mnt/dev/dirceusilva/dados/Cover/SHS100k'
#direct = '/home/dirceusilva/Documents/Test'

all_files = glob.glob(os.path.join(direct,'**/*.wav'))

for k,f in enumerate(all_files):
    print("{:d} - {:d} - {} ".format(k,len(all_files), f))
    
    fileout = f[:-3]+ "mp3"

    !lame --silent {f} {fileout}
    if os.path.exists(fileout):
        #print("converted")
        os.remove(f)
    



## List Failed  Downloads

In [ ]:
filelist = "list.csv"
outdir = '/mnt/dataset/public/covers/SHS100k'


In [ ]:
list_songs_df = pd.read_csv(filelist, sep='\t', names=["set_id", "ver_id", "title", "performer", "url", "status"], header=None)


In [ ]:
index_failed=[]

for k in range(len(list_songs_df)):
    row = list_songs_df.iloc[k]
    print("{:d} de {:d} - {} - {}".format(k, len(list_songs_df), row["set_id"], row["ver_id"]))
    dirout = os.path.join(outdir, str(row["set_id"])) 

    filepath = os.path.join(dirout,str(row["ver_id"])+ '.mp3')
    
    if not os.path.exists(filepath):
        try:
            os.mkdir(dirout)
        except:
            print("Creation of the directory {} failed".format(row["set_id"]))
            
        name = str(row["ver_id"])+ '.%(ext)s'
        if not download_clip(row["url"], name, dirout):
            index_failed.append((k, row["url"]))


### Save List of Failed Downloads

In [ ]:
index = []
for k, url_y in index_failed:
    index.append(k)

In [ ]:
df_1 = list_songs_df.iloc[index]

In [ ]:
df_1.to_csv("list_failed.csv", sep=";", header=False)